In [13]:
from math import exp, sqrt
from scipy.special import erf

def calibrateAnalyticGaussianMechanism(epsilon, delta, GS, tol = 1.e-12):
    """ Calibrate a Gaussian perturbation for differential privacy using the analytic Gaussian mechanism of [Balle and Wang, ICML'18]
    Arguments:
    epsilon : target epsilon (epsilon > 0)
    delta : target delta (0 < delta < 1)
    GS : upper bound on L2 global sensitivity (GS >= 0)
    tol : error tolerance for binary search (tol > 0)
    Output:
    sigma : standard deviation of Gaussian noise needed to achieve (epsilon,delta)-DP under global sensitivity GS
    """

    def Phi(t):
        return 0.5*(1.0 + erf(float(t)/sqrt(2.0)))

    def caseA(epsilon,s):
        return Phi(sqrt(epsilon*s)) - exp(epsilon)*Phi(-sqrt(epsilon*(s+2.0)))

    def caseB(epsilon,s):
        return Phi(-sqrt(epsilon*s)) - exp(epsilon)*Phi(-sqrt(epsilon*(s+2.0)))

    def doubling_trick(predicate_stop, s_inf, s_sup):
        while(not predicate_stop(s_sup)):
            s_inf = s_sup
            s_sup = 2.0*s_inf
        return s_inf, s_sup

    def binary_search(predicate_stop, predicate_left, s_inf, s_sup):
        s_mid = s_inf + (s_sup-s_inf)/2.0
        while(not predicate_stop(s_mid)):
            if (predicate_left(s_mid)):
                s_sup = s_mid
            else:
                s_inf = s_mid
            s_mid = s_inf + (s_sup-s_inf)/2.0
        return s_mid

    delta_thr = caseA(epsilon, 0.0)

    if (delta == delta_thr):
        alpha = 1.0

    else:
        if (delta > delta_thr):
            predicate_stop_DT = lambda s : caseA(epsilon, s) >= delta
            function_s_to_delta = lambda s : caseA(epsilon, s)
            predicate_left_BS = lambda s : function_s_to_delta(s) > delta
            function_s_to_alpha = lambda s : sqrt(1.0 + s/2.0) - sqrt(s/2.0)

        else:
            predicate_stop_DT = lambda s : caseB(epsilon, s) <= delta
            function_s_to_delta = lambda s : caseB(epsilon, s)
            predicate_left_BS = lambda s : function_s_to_delta(s) < delta
            function_s_to_alpha = lambda s : sqrt(1.0 + s/2.0) + sqrt(s/2.0)

        predicate_stop_BS = lambda s : abs(function_s_to_delta(s) - delta) <= tol

        s_inf, s_sup = doubling_trick(predicate_stop_DT, 0.0, 1.0)
        s_final = binary_search(predicate_stop_BS, predicate_left_BS, s_inf, s_sup)
        alpha = function_s_to_alpha(s_final)
        
    sigma = alpha*GS/sqrt(2.0*epsilon)

    return sigma

In [14]:
import numpy as np
sigmas = []
for epsilon in np.linspace(0.1,10,100):
    sigmas.append(calibrateAnalyticGaussianMechanism(epsilon,1e-5,1))

In [15]:
sigmas

[30.74956598384496,
 16.30413344124468,
 11.238044475237771,
 8.629573593950482,
 7.031826674729583,
 5.949578915764334,
 5.166492509773933,
 4.572761745539473,
 4.1066243390331225,
 3.730631634944469,
 3.420734330167215,
 3.16076979479468,
 2.9394675983979295,
 2.748724797516329,
 2.582563708274324,
 2.4364766046696285,
 2.306998838851466,
 2.191423048377722,
 2.087602437326611,
 1.9938124430242377,
 1.9086516325061702,
 1.8309691515971256,
 1.7598109604713474,
 1.6943794438569728,
 1.6340024958670345,
 1.578109698314794,
 1.5262137420299624,
 1.4778958191274956,
 1.4327939032839667,
 1.3905934534820914,
 1.3510198638082505,
 1.3138322799020332,
 1.2788185453565706,
 1.2457910964230514,
 1.2145833991346378,
 1.1850471734874453,
 1.157049870029199,
 1.1304726880154687,
 1.1052087826145394,
 1.0811618535469825,
 1.0582448077742248,
 1.036378733742674,
 1.0154919386878676,
 0.9955191326404649,
 0.976400739853415,
 0.9580822793241547,
 0.940513820225213,
 0.9236495463196214,
 0.9074473071